# Stage 2 Training on Google Colab

完整的 Colab 遠程訓練流程

費用：**完全免費** (GPU 加速)
耗時：約 **5-8 分鐘**

## 使用說明

1. 點擊上方 **Open in Colab** 按鍵
2. 或複製 Notebook 連結到 Colab
3. 依序運行所有 Cell
4. 結果自動保存到 Google Drive

## 環境設置

In [ ]:
# 檢查 Colab 環境
import sys
try:
    from google.colab import drive
    IN_COLAB = True
    print('✓ Running on Google Colab')
except ImportError:
    IN_COLAB = False
    print('⚠ Not running on Google Colab (local execution)')

# 檢查 GPU
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f'✓ GPU Available: {[gpu.name for gpu in gpus]}')
else:
    print('⚠ No GPU detected')

In [ ]:
# 安裝必要的套件
print('Installing dependencies...')

!pip install -q lightgbm huggingface-hub scikit-learn pandas numpy tensorflow

print('✓ All dependencies installed')

In [ ]:
# 掛載 Google Drive（可選）
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print('✓ Google Drive mounted at /content/drive')
    DRIVE_ROOT = '/content/drive/MyDrive'
else:
    DRIVE_ROOT = '.'
    print('⚠ Not using Google Drive')

In [ ]:
# 克隆 GitHub 倉庫
import os
import subprocess

REPO_URL = 'https://github.com/caizongxun/crypto-zigzag-ml.git'
REPO_DIR = '/content/crypto-zigzag-ml'

if not os.path.exists(REPO_DIR):
    print(f'Cloning repository from {REPO_URL}...')
    os.system(f'git clone {REPO_URL} {REPO_DIR}')
    print('✓ Repository cloned')
else:
    print(f'✓ Repository already exists at {REPO_DIR}')
    print('  Pulling latest changes...')
    os.chdir(REPO_DIR)
    os.system('git pull')

os.chdir(REPO_DIR)
print(f'Current directory: {os.getcwd()}')

## Stage 2 訓練（主要流程）

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 配置
SYMBOL = 'BTCUSDT'
TIMEFRAME = '15m'
SYMBOL_SHORT = 'BTC'
HF_DATASET_ID = 'zongowo111/v2-crypto-ohlcv-data'
HF_MODEL_PATH = f'v1_model/{SYMBOL}/{TIMEFRAME}'
HF_DATA_PATH = f'klines/{SYMBOL}/{SYMBOL_SHORT}_{TIMEFRAME}.parquet'

# Stage 1 Model Architecture Parameters
STAGE1_SEQUENCE_LENGTH = 10
STAGE1_NUM_FEATURES = 20

# 本地路徑
DATA_DIR = Path('data')
MODEL_DIR = Path('models')
STAGE1_MODEL_DIR = MODEL_DIR / 'stage1' / f'{SYMBOL.lower()}_{TIMEFRAME}'
STAGE2_DATA_DIR = DATA_DIR / 'stage2' / f'{SYMBOL.lower()}_{TIMEFRAME}'
STAGE2_MODEL_DIR = MODEL_DIR / 'stage2' / f'{SYMBOL.lower()}_{TIMEFRAME}'

# 建立目錄
STAGE1_MODEL_DIR.mkdir(parents=True, exist_ok=True)
STAGE2_DATA_DIR.mkdir(parents=True, exist_ok=True)
STAGE2_MODEL_DIR.mkdir(parents=True, exist_ok=True)

print(f'Symbol: {SYMBOL}')
print(f'Timeframe: {TIMEFRAME}')
print(f'Stage 1 Input Shape: ({STAGE1_SEQUENCE_LENGTH}, {STAGE1_NUM_FEATURES})')
print(f'Stage 1 Model Dir: {STAGE1_MODEL_DIR}')
print(f'Stage 2 Model Dir: {STAGE2_MODEL_DIR}')

In [ ]:
# Step 1: 從 HuggingFace 下載 Stage 1 模型
from huggingface_hub import hf_hub_download
from tensorflow import keras

print(f'[1/7] Downloading Stage 1 model from HuggingFace...')
print(f'Dataset: {HF_DATASET_ID}')
print(f'Path: {HF_MODEL_PATH}')

# 下載 classification.h5
try:
    classification_path = hf_hub_download(
        repo_id=HF_DATASET_ID,
        filename=f'{HF_MODEL_PATH}/classification.h5',
        repo_type='dataset',
        cache_dir=str(STAGE1_MODEL_DIR)
    )
    print(f'✓ Downloaded classification.h5')
except Exception as e:
    print(f'✗ Error: {e}')
    raise

# 加載模型
classification_files = list(STAGE1_MODEL_DIR.rglob('classification.h5'))
stage1_model_path = classification_files[0]
stage1_model = keras.models.load_model(str(stage1_model_path))

# Get input shape
input_shape = stage1_model.input_shape
if len(input_shape) == 3:
    STAGE1_SEQUENCE_LENGTH = input_shape[1]
    STAGE1_NUM_FEATURES = input_shape[2]
    print(f'✓ Stage 1 model loaded')
    print(f'  Input shape: {input_shape}')
else:
    raise ValueError(f'Expected 3D input, got {input_shape}')

In [ ]:
# Step 2: 從 HuggingFace 下載訓練數據
print(f'\n[2/7] Downloading training data from HuggingFace...')
print(f'File: {HF_DATA_PATH}')

try:
    data_file = hf_hub_download(
        repo_id=HF_DATASET_ID,
        filename=HF_DATA_PATH,
        repo_type='dataset',
        cache_dir=str(DATA_DIR)
    )
    print(f'✓ Downloaded data')
    data_file = Path(data_file)
except Exception as e:
    print(f'✗ Error downloading: {e}')
    raise

# 加載數據
df = pd.read_parquet(data_file)
print(f'✓ Data loaded: {df.shape[0]:,} rows, {df.shape[1]} columns')

In [ ]:
# Step 3: 特徵工程和 ZigZag 標籤
print(f'\n[3/7] Feature engineering...')

sys.path.insert(0, os.path.abspath('.'))
from src.zigzag_indicator import ZigZagIndicator
from src.features import FeatureEngineer
from src.utils import time_series_split

# 應用 ZigZag
zigzag = ZigZagIndicator(depth=12, deviation=5, backstep=2)
df = zigzag.label_kbars(df)
print(f'✓ ZigZag labels applied')

# 計算技術指標
fe = FeatureEngineer(lookback_periods=[5, 10, 20, 50, 200])
df = fe.calculate_all_features(df)

# 獲取特徵列
feature_cols = fe.get_feature_columns(df)
if 'symbol' in feature_cols:
    feature_cols.remove('symbol')

# 處理缺失值
df[feature_cols] = df[feature_cols].fillna(method='ffill').fillna(0)
print(f'✓ Features calculated: {len(feature_cols)} features')

In [ ]:
# Step 4: Create 3D Time Series Features
print(f'\n[4/7] Data splitting and Stage 1 filtering...')

def create_3d_sequences(X, y, seq_length=10):
    X_seq = []
    y_seq = []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)

# Split data
train_df, val_df, test_df = time_series_split(df, train_ratio=0.7, validation_ratio=0.15)

# === Train Set ===
X_train_2d = train_df[feature_cols].values
y_train_2d = train_df['zigzag_label'].values
X_train_3d, y_train_3d = create_3d_sequences(X_train_2d, y_train_2d, seq_length=STAGE1_SEQUENCE_LENGTH)
print(f'  Train 3D shape: {X_train_3d.shape}')

# === Validation Set ===
X_val_2d = val_df[feature_cols].values
y_val_2d = val_df['zigzag_label'].values
X_val_3d, y_val_3d = create_3d_sequences(X_val_2d, y_val_2d, seq_length=STAGE1_SEQUENCE_LENGTH)
print(f'  Val 3D shape: {X_val_3d.shape}')

# === Test Set ===
X_test_2d = test_df[feature_cols].values
y_test_2d = test_df['zigzag_label'].values
X_test_3d, y_test_3d = create_3d_sequences(X_test_2d, y_test_2d, seq_length=STAGE1_SEQUENCE_LENGTH)
print(f'  Test 3D shape: {X_test_3d.shape}')

print(f'✓ All sets converted to 3D format')

In [ ]:
# Step 5: Apply Stage 1 Model
print(f'\nApplying Stage 1 model to filtered datasets...')

# === Train Set ===
stage1_probs_train = stage1_model.predict(X_train_3d, verbose=0)
stage1_preds_train = (stage1_probs_train[:, 1] > 0.5).astype(int)
signal_mask = stage1_preds_train == 1
X_stage2_train_3d = X_train_3d[signal_mask]
y_stage2_train = y_train_3d[signal_mask]
X_stage2_train = X_stage2_train_3d[:, -1, :]
valid_mask = y_stage2_train > 0
X_stage2_train = X_stage2_train[valid_mask]
y_stage2_train = y_stage2_train[valid_mask]
print(f'  Train: {len(X_stage2_train):,} samples')

# === Validation Set ===
stage1_probs_val = stage1_model.predict(X_val_3d, verbose=0)
stage1_preds_val = (stage1_probs_val[:, 1] > 0.5).astype(int)
signal_mask_val = stage1_preds_val == 1
X_stage2_val_3d = X_val_3d[signal_mask_val]
y_stage2_val = y_val_3d[signal_mask_val]
X_stage2_val = X_stage2_val_3d[:, -1, :]
valid_mask_val = y_stage2_val > 0
X_stage2_val = X_stage2_val[valid_mask_val]
y_stage2_val = y_stage2_val[valid_mask_val]
print(f'  Val: {len(X_stage2_val):,} samples')

# === Test Set ===
stage1_probs_test = stage1_model.predict(X_test_3d, verbose=0)
stage1_preds_test = (stage1_probs_test[:, 1] > 0.5).astype(int)
signal_mask_test = stage1_preds_test == 1
X_stage2_test_3d = X_test_3d[signal_mask_test]
y_stage2_test = y_test_3d[signal_mask_test]
X_stage2_test = X_stage2_test_3d[:, -1, :]
valid_mask_test = y_stage2_test > 0
X_stage2_test = X_stage2_test[valid_mask_test]
y_stage2_test = y_stage2_test[valid_mask_test]
print(f'  Test: {len(X_stage2_test):,} samples')

In [ ]:
# Step 6: 保存 Stage 2 訓練數據
print(f'\n[5/7] Saving Stage 2 training data...')

with open(STAGE2_DATA_DIR / 'X_stage2_train.pkl', 'wb') as f:
    pickle.dump(X_stage2_train, f)
with open(STAGE2_DATA_DIR / 'y_stage2_train.pkl', 'wb') as f:
    pickle.dump(y_stage2_train, f)
with open(STAGE2_DATA_DIR / 'X_stage2_val.pkl', 'wb') as f:
    pickle.dump(X_stage2_val, f)
with open(STAGE2_DATA_DIR / 'y_stage2_val.pkl', 'wb') as f:
    pickle.dump(y_stage2_val, f)
with open(STAGE2_DATA_DIR / 'X_stage2_test.pkl', 'wb') as f:
    pickle.dump(X_stage2_test, f)
with open(STAGE2_DATA_DIR / 'y_stage2_test.pkl', 'wb') as f:
    pickle.dump(y_stage2_test, f)

print(f'✓ Data saved to {STAGE2_DATA_DIR}')

In [ ]:
# Step 7: 訓練 Stage 2 模型
print(f'\n[6/7] Training Stage 2 model...')

from src.stage2_trainer import Stage2Trainer

trainer = Stage2Trainer(model_dir=str(STAGE2_MODEL_DIR))

train_results = trainer.train(
    X_stage2_train, y_stage2_train,
    X_stage2_val, y_stage2_val,
    normalize=True,
    cv_folds=5,
    save_model=True
)

print(f'✓ Model trained')
print(f'  Train Accuracy: {train_results["train_accuracy"]:.4f}')
print(f'  Val Accuracy: {train_results["val_accuracy"]:.4f}')

In [ ]:
# Step 8: 評估和交叉驗證
print(f'\n[7/7] Evaluation and cross-validation...')

test_metrics = trainer.evaluate(X_stage2_test, y_stage2_test)

cv_results = trainer.cross_validate(
    np.vstack([X_stage2_train, X_stage2_val]),
    np.hstack([y_stage2_train, y_stage2_val]),
    cv=5
)

print(f'✓ Evaluation complete')

## 結果總結

In [ ]:
print(f'\n' + '='*80)
print(f'STAGE 2 TRAINING COMPLETE - {SYMBOL} {TIMEFRAME}')
print(f'='*80)

print(f'\n📊 DATA STATISTICS:')
print(f'  Original K-bars: {len(df):,}')
print(f'  Stage 1 Signals: {signal_mask.sum() + signal_mask_val.sum() + signal_mask_test.sum():,}')
print(f'  Stage 2 Valid Samples: {len(X_stage2_train) + len(X_stage2_val) + len(X_stage2_test):,}')

print(f'\n📈 TRAIN/VAL/TEST SPLIT:')
print(f'  Train: {len(X_stage2_train):,}')
print(f'  Val: {len(X_stage2_val):,}')
print(f'  Test: {len(X_stage2_test):,}')

print(f'\n🎯 MODEL PERFORMANCE:')
print(f'  Train Accuracy: {train_results["train_accuracy"]:.4f}')
print(f'  Val Accuracy: {train_results["val_accuracy"]:.4f}')
print(f'  Test Accuracy: {test_metrics["accuracy"]:.4f}')
print(f'  Test F1-Score: {test_metrics["f1_score"]:.4f}')

print(f'\n✅ CROSS-VALIDATION:')
print(f'  Mean Accuracy: {cv_results["mean_accuracy"]:.4f}')
print(f'  Std Accuracy: {cv_results["std_accuracy"]:.4f}')

print(f'\n💾 MODELS SAVED:')
for file in sorted(STAGE2_MODEL_DIR.glob('*')):
    if file.is_file():
        size = file.stat().st_size / 1024 / 1024
        print(f'  {file.name} ({size:.2f} MB)')

print(f'\n' + '='*80)

## 保存結果到 Google Drive

In [ ]:
if IN_COLAB:
    import shutil
    from google.colab import drive
    
    print('Saving results to Google Drive...')
    
    # 建立 Colab 結果目錄
    colab_results = Path(DRIVE_ROOT) / 'Colab Results' / 'Stage2' / f'{SYMBOL}_{TIMEFRAME}'
    colab_results.mkdir(parents=True, exist_ok=True)
    
    # 複製模型檔案
    for model_file in STAGE2_MODEL_DIR.glob('*'):
        if model_file.is_file():
            shutil.copy(model_file, colab_results / model_file.name)
            print(f'  ✓ Copied {model_file.name}')
    
    # 複製訓練數據
    data_subdir = colab_results / 'training_data'
    data_subdir.mkdir(parents=True, exist_ok=True)
    for data_file in STAGE2_DATA_DIR.glob('*.pkl'):
        shutil.copy(data_file, data_subdir / data_file.name)
        print(f'  ✓ Copied {data_file.name}')
    
    print(f'\n✓ Results saved to: {colab_results}')
else:
    print('Not running on Colab - skipping Google Drive save')